In [85]:
class RNA:
    #Objects
    i = 'pute'
    #none so far
    #Methods
    def get_coefficients(self,path_to_genbank,path_to_model,transcriptomic,
                             CELL_WEIGHT = 0.15,TOTAL_RNA_RATIO=0.2,
                             rRNA_RATIO=0.9,tRNA_RATIO=0.05,mRNA_RATIO=0.05):
        #Imports
        import pandas as pd
        
        #Constant
        BASES = ['A','U','C','G']
        
        def import_genome(path_to_genbank):
            #This function import the genome record from genbank
            from Bio import SeqIO
            genome_record = SeqIO.read(path_to_genbank,'genbank')
            return genome_record
        
        def import_model(path_to_model):
            import cobra
            extension = path_to_model.split('.')[-1]
            if extension == 'json':
                model = cobra.io.load_json_model(path_to_model)
            elif extension == 'xml':
                model = cobra.io.read_sbml_model(path_to_model)
            return model
       
        def get_number(seq):
            #This function is used if and only if the level of expression of each gene is assumed equal
            ratio_gene = {'A':0,'U':0,'C':0,'G':0}
            for base in BASES:
                number = float(seq.count(base))
                ratio_gene[base] = number
            return ratio_gene

        def get_fraction(seq):
            #This function is to be used in case the level of expression of each gene cannot be assumed equal
            ratio_gene = {'A':0,'U':0,'C':0,'G':0}
            for base in BASES:
                fraction = float(seq.count(base))/len(seq)
                ratio_gene[base] = fraction
            return ratio_gene
        
        def get_RNA_sequence(location,strand):
            #This function spits out the RNA sequence of a given gene
            from Bio.Alphabet import IUPAC
            from Bio.Seq import Seq
            #Get the gene sequence
            sequence = []
            for number in range(location.start, location.end):
                sequence.append(genome_record[number])
            seq_str = ''.join(sequence)
            my_seq = Seq(seq_str, IUPAC.unambiguous_dna)
            try:
                if strand == 1:
                    rna_seq = my_seq.transcribe()
                elif strand == -1:
                    rna_seq = my_seq.reverse_complement().transcribe()
            except:
                raise('error no strand provided')

            return str(rna_seq)

        def make_number_df(number_list,locus_list, seq_list):
            #This function gnerates a dataframe with locus info, sequence and amount of each base
            A_number,U_number, C_number, G_number = [],[],[],[]
            for d in number_list:
                for base in BASES:
                    value = d.get(base)
                    if base == 'A':
                        A_number.append(value)
                    elif base == 'U':
                        U_number.append(value)
                    elif base == 'C':
                        C_number.append(value)
                    elif base == 'G':
                        G_number.append(value)

            generic_df = pd.DataFrame({'locus':locus_list,'sequence':seq_list,
                                              'A':A_number,'U':U_number,
                                              'C':C_number,'G':G_number},
                                      columns = ['locus','sequence','A','U','C','G']) 

            return generic_df 
        
        def get_total_fractions(df):
            total_per_base = df.sum(axis=0, numeric_only=True)
            grand_total = float(total_per_base.sum(axis=0))

            fraction_dict = {'A':0,'U':0,'G':0,'C':0}
            fraction_dict['A'] = total_per_base.A/grand_total
            fraction_dict['U']= total_per_base.U/grand_total
            fraction_dict['G'] = total_per_base.G/grand_total
            fraction_dict['C'] = total_per_base.C/grand_total
            return fraction_dict
        
        def get_mRNA_fractions(df,transcriptomic):
            #Merge dataframes
            mean_abundance = pd.merge(left=df, right=transcriptomic, left_on='locus',right_on='gene_ID')
            #Generate list of normalized values per gene per base by RPKM from transcriptomic data
            A_norm,U_norm,G_norm,C_norm = [],[],[],[]
            for i,row in mean_abundance.iterrows():
                #1- Multiply fraction by abundance for each gene
                A_mean = row.A*row.Mean
                U_mean = row.U*row.Mean
                G_mean = row.G*row.Mean
                C_mean = row.C*row.Mean
                A_norm.append(A_mean)
                U_norm.append(U_mean)
                G_norm.append(G_mean)
                C_norm.append(C_mean)

            #Get the mean of each list stored in a dictionary
            mRNA_fractions = {'A':0,'U':0,'C':0,'G':0}
            mRNA_fractions['A'] = sum(A_norm)/sum(mean_abundance.Mean)
            mRNA_fractions['U'] = sum(U_norm)/sum(mean_abundance.Mean)
            mRNA_fractions['G'] = sum(G_norm)/sum(mean_abundance.Mean)
            mRNA_fractions['C'] = sum(C_norm)/sum(mean_abundance.Mean)
            
            return mRNA_fractions
        
        def process_record(genome_record):
            rRNA_seq, rRNA_locus,rRNA_number = [],[],[]
            tRNA_seq, tRNA_locus,tRNA_number = [],[],[]
            mRNA_seq, mRNA_locus,mRNA_fraction = [],[],[]
            
            for element in genome_record.features:
                if element.type == 'CDS':
                    rna_seq = get_RNA_sequence(element.location,element.strand)
                    mRNA_seq.append(rna_seq)
                    mRNA_locus.append(element.qualifiers['locus_tag'][0])
                    mRNA_fraction.append(get_fraction(rna_seq))
                    
                if element.type == 'tRNA':
                    rna_seq = get_RNA_sequence(element.location,element.strand)
                    tRNA_seq.append(rna_seq)
                    tRNA_locus.append(element.qualifiers['locus_tag'][0])
                    tRNA_number.append(get_number(rna_seq))
                if element.type == 'rRNA':
                    rna_seq = get_RNA_sequence(element.location,element.strand)
                    rRNA_seq.append(rna_seq)
                    rRNA_locus.append(element.qualifiers['locus_tag'][0])
                    rRNA_number.append(get_number(rna_seq))
            #Make dataframe for each
            rRNA_dict = get_total_fractions(make_number_df(rRNA_number,rRNA_locus,rRNA_seq))
            tRNA_dict = get_total_fractions(make_number_df(tRNA_number,tRNA_locus,tRNA_seq))
            mRNA_dict = get_mRNA_fractions(make_number_df(mRNA_fraction,mRNA_locus,mRNA_seq),transcriptomic)
            
            return rRNA_dict, tRNA_dict, mRNA_dict
        
        def total_coefficients(mRNA_fractions,tRNA_fractions,rRNA_fractions):
            #Multiply mRNA,rRNA and tRNA 
            #dict values by the coefficients 
            #to get the average of each base from RNA in the cell
            RNA_total = {'A':0,'U':0,'C':0,'G':0}
            for letter in BASES:
                mrna = mRNA_fractions[letter]*mRNA_RATIO
                trna = tRNA_fractions[letter]*tRNA_RATIO
                rrna = rRNA_fractions[letter]*rRNA_RATIO
                RNA_total[letter] = mrna + trna + rrna 

            return RNA_total
        
        def convert_to_mmolgDW(RNA_coefficients,model,RNA_RATIO,CELL_WEIGHT):
            #Get coefficients for BIOMASS
            #Transform the ratios into mmol/gDW
            RNA_WEIGHT = CELL_WEIGHT*RNA_RATIO

            rna_base_to_bigg = {'A':model.metabolites.atp_c,'U':model.metabolites.utp_c,
                            'C':model.metabolites.ctp_c,'G':model.metabolites.gtp_c}
            RNA_biomass_ratios = {'A':0,'U':0,'C':0,'G':0}
            #Get the total weight of each letter
            for letter in BASES:
                ratio = RNA_coefficients.get(letter)
                total_weight = ratio*RNA_WEIGHT
                metab = rna_base_to_bigg.get(letter)
                mol_weight = metab.formula_weight
                mmols_per_cell = (total_weight/mol_weight)*1000
                mmols_per_gDW = mmols_per_cell/CELL_WEIGHT
                RNA_biomass_ratios[letter] = mmols_per_gDW
                
            return RNA_biomass_ratios
            
        #Operations
        genome_record = import_genome(path_to_genbank)
        rRNA_dict, tRNA_dict, mRNA_dict = process_record(genome_record)
        RNA_coefficients = total_coefficients(rRNA_dict, tRNA_dict, mRNA_dict)
        RNA_biomass_ratios = convert_to_mmolgDW(RNA_coefficients, 
                                                import_model(path_to_model),TOTAL_RNA_RATIO,CELL_WEIGHT)
        
        return RNA_biomass_ratios
        

In [89]:
rna = RNA()
RNA.get_coefficients(rna,genbank_path,model_path,transcriptomic,CELL_WEIGHT=0.25,TOTAL_RNA_RATIO=0.2)

{'A': 0.15532006950216207,
 'C': 0.057627642295224261,
 'G': 0.069625841327269289,
 'U': 0.1210012748162293}

In [91]:
import pandas as pd
raw_file = pd.read_table('/home/jean-christophe/Documents/Maitrise_UCSD/Experimental_validation/Transcriptomics/florum_gene_expression_all_rep.txt')
transcriptomic = raw_file[['gene_ID','Mean']].copy()
genbank_path = '/home/jean-christophe/Documents/Maitrise_UCSD/GenBank_files/Mflorum.gbff'
model_path = '/home/jean-christophe/Documents/Maitrise_UCSD/Modelling/Mesoplasma_florum/Functionnal_testing/finalMeso1.json'
transcriptomic.to_csv('/home/jean-christophe/Documents/Maitrise_UCSD/biomass/transcriptomic.csv')

In [50]:
class dummy:
    
    def __init__(self):
        return None
    def print_dummy(self):
        print('hello dummy',self)

In [51]:
i = dummy()

i.print_dummy()

('hello dummy', <__main__.dummy instance at 0x7f80f4a533f8>)


In [24]:
for element in genome_record.features:
    print(element)

type: source
location: [0:793224](+)
qualifiers:
    Key: db_xref, Value: ['ATCC:33453', 'taxon:265311']
    Key: mol_type, Value: ['genomic DNA']
    Key: organism, Value: ['Mesoplasma florum L1']
    Key: strain, Value: ['L1; ATCC 33453']

type: gene
location: [0:1332](+)
qualifiers:
    Key: db_xref, Value: ['GeneID:2897662']
    Key: gene, Value: ['dnaA']
    Key: locus_tag, Value: ['Mfl001']

type: CDS
location: [0:1332](+)
qualifiers:
    Key: codon_start, Value: ['1']
    Key: db_xref, Value: ['GI:50364816', 'GeneID:2897662']
    Key: gene, Value: ['dnaA']
    Key: locus_tag, Value: ['Mfl001']
    Key: note, Value: ['binds to the dnaA-box as an ATP-bound complex at the origin of replication during the initiation of chromosomal replication; can also affect transcription of multiple genes including itself.']
    Key: product, Value: ['chromosome replication initiator DnaA']
    Key: protein_id, Value: ['YP_053241.1']
    Key: transl_table, Value: ['4']
    Key: translation, Value: